In [1]:
import numpy as np
import re
from keras.models import Sequential, Model
from keras.layers import Dense, Masking
from keras.layers import LSTM
from keras.layers import Input
from keras.utils import np_utils

In [ ]:

Lalphabet = ["Once", "upon", "a", "time,", "in", "a", "far-off", "kingdom,", "there", "was", "a", "kind", "and", "noble", "king!", "who", "ruled", "over", "a", "peaceful", "and", "prosperous", "land.", "One", "day,", "an", "evil", "sorceress", "appeared", "and", "cast", "a", "spell", "on", "the", "king,", "causing", "him", "to", "fall", "into", "a", "deep", "sleep.", "The", "sorceress", "declared", "that", "the", "only", "way", "to", "break", "the", "spell", "was", "for", "a", "brave", "and", "courageous", "hero", "to", "journey", "to", "the", "top", "of", "a", "tall,", "treacherous", "mountain", "and", "retrieve", "a", "magical", "flower.", "Many", "brave", "knights", "tried", "to", "undertake", "the", "quest,", "but", "none", "were", "successful.", "The", "mountain", "was", "filled", "with", "dangerous", "creatures", "and", "perilous", "obstacles,", "and", "many", "knights", "never", "returned.", "One", "day,", "a", "young", "squire", "named", "Jack", "decided", "to", "attempt", "the", "quest.", "He", "had", "always", "been", "fascinated", "by", "tales", "of", "magic", "and", "adventure,", "and", "he", "was", "determined", "to", "be", "the", "one", "to", "break", "the", "spell", "and", "save", "the", "king.", "Despite", "the", "warnings", "of", "his", "friends", "and", "family,", "Jack", "set", "out", "on", "his", "journey,", "armed", "with", "only", "a", "sword", "and", "a", "shield.", "After", "many", "trials", "and", "tribulations,", "Jack", "finally", "reached", "the", "top", "of", "the", "mountain", "and", "plucked", "the", "magical", "flower.", "He", "brought", "it", "back", "to", "the", "kingdom,", "and", "when", "the", "sorceress", "saw", "it,", "she", "knew", "that", "the", "spell", "had", "been", "broken.", "The", "king", "awoke", "from", "his", "slumber,", "and", "the", "kingdom", "rejoiced.", "Jack", "was", "hailed", "as", "a", "hero,", "and","X"]
palphabet= []
for i in Lalphabet:
  palphabet.append(i.lower())

split_alphabet = []
for word in palphabet:
    if word[-1] in [',',".","!","?"]:
        split_alphabet.append(word[0:-1])
        split_alphabet.append(word[-1])
    else:
      split_alphabet.append(word)
print(split_alphabet)

alphabet = split_alphabet

In [ ]:
# define the raw dataset
tt = ["Hej","med","dig","jeg","hedder","Tristan","og","dør","ofte","i","D&D"]
sentence = list(set(alphabet))
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

In [ ]:
# Define the sequence lengths to use
seq_lengths = [1, 2, 3]
max_length = max(seq_lengths)

# Prepare the dataset of input to output pairs encoded as integers
dataX = []
dataY = []
for seq_length in seq_lengths:
    for i in range(0, len(alphabet) - seq_length, 1):
        seq_in = alphabet[i:i + seq_length]
        seq_out = alphabet[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])

# Calculate the number of samples in dataX
num_samples = len(dataX)

# Reshape the input data
X = np.zeros((num_samples, max_length, 1), dtype=np.float32)
for i, x in enumerate(dataX):
    for j, c in enumerate(x):
        X[i, j, 0] = c

# Normalize the input data
X /= float(len(alphabet))

# One-hot encode the output data
y = np_utils.to_categorical(dataY)

In [ ]:
# Define the input layer
inputs = Input(shape=(None, 1))  # Accept sequences of any length

# Add a masking layer to ignore padded values
mask = Masking(mask_value="X")(inputs)

# Define the LSTM layer
lstm = LSTM(32)(mask)

# Define the output layer
outputs = Dense(y.shape[1], activation='softmax')(lstm)

# Create the model
model = Model(inputs, outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100)


In [ ]:
# summarize performance of the mode
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# demonstrate some model predictions
for pattern in dataX:
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

In [ ]:
# Encode the input sequence as integers
input_sequence = ['the',"spell"]
input_encoded = [char_to_int[word] for word in input_sequence]

# Reshape the input into a 2D array
input_array = np.reshape(input_encoded, (1, len(input_sequence), 1))

# Normalize the input
input_array = input_array / float(len(alphabet))

# Predict the next word in the sequence
prediction = model.predict(input_array)

# Find the index of the word with the highest probability
predicted_word_index = np.argmax(prediction)

# Decode the predicted word
predicted_word = int_to_char[predicted_word_index]

print("Input sequence:", input_sequence)
print("Predicted next word:", predicted_word)